In [2]:
import torch
from torch import nn
from datasets import load_dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from torch.optim.lr_scheduler import ReduceLROnPlateau
# from sklearnex import patch_sklearn
# patch_sklearn()
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
import gc
import json
import pickle
import numpy as np    
import pandas as pd
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2024-04-18 15:43:17.594052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 15:43:17.594162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 15:43:17.744208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/m3-models/model_3_15.pth
/kaggle/input/m3-models/model_3_20.pth
/kaggle/input/m3-models/tokenizerM3.pth
/kaggle/input/m3-models/model_3_10.pth
/kaggle/input/m3-models/model_3_25.pth
/kaggle/input/m3-models/model_3_5.pth
/kaggle/input/m3-models/modelM3.pth
/kaggle/input/m3-models/__results___files/__results___28_0.png
/kaggle/input/emotion-dataset/train_file.json
/kaggle/input/emotion-dataset/val_file.json


In [3]:
PATH="/kaggle/input/emotion-dataset/"
OUTPATH='/kaggle/working/'
MODEL_PATH = "/kaggle/input/m3-models/"
# PATH="D:\\ghd\\NLP-Assignments\\Assignment4\\data\\"
# PATH="data/"
# OUTPATH='output/'
BATCH_SIZE=512
MAX_LENGTH=256
MAX_UTTERANCES=24
ROBERTA_LABELS=100
EPOCHS=25
EOS='</s>'
SEP='[SEP]'
# SOS='o'

torch.manual_seed(0)

model_id = "roberta-base"

In [4]:
device = ""
if torch.cuda.is_available():
    device = "cuda"
    print(torch.cuda.get_device_name(0))
else:
    device = "cpu"

device = torch.device(device)
device

Tesla P100-PCIE-16GB


device(type='cuda')

In [5]:
pad=-100
def get_train(data):
    x=[]
    y=[]
    for row in data:
        speakers=row['speakers']
        emotions=row['emotions']
        utterances=row['utterances']
        triggers=row['triggers']
        # print(speakers,emotions,utterances)
        # trim in front
        while len(utterances)>MAX_UTTERANCES:
            utterances.pop(0)
            speakers.pop(0)
            emotions.pop(0)
            triggers.pop(0)
        
        # print(speakers,emotions,utterances)
        # pad
        while len(utterances)<MAX_UTTERANCES:
            utterances.insert(0,EOS)
            speakers.insert(0,EOS)
            emotions.insert(0,EOS)
            triggers.insert(0,-100)
        # print(speakers[1],emotions[1],utterances[1])
        t=""
        s1=[]
        s2=[]        
        for i in range(MAX_UTTERANCES):
            if(utterances[i]==EOS):
                s1.append(utterances[i])
                s2.append(utterances[i])
                t=EOS
                continue
            if(t==EOS):
                t=""
            # text.append([t,f"{speakers[-1]}:{utterances[-1]}:{emotions[-1]}"])
            s1.append(t)
            s2.append(f"{speakers[-1]}:{utterances[-1]}:{emotions[-1]}")

            t+=f"{speakers[i]}:{utterances[i]}:{emotions[i]} "
        x.append([s1,s2])
        trigs=[]
        for i in triggers:
            if(i is None):
                trigs.append(0)
            else:
                trigs.append(int(i))
        y.append(trigs)
    
    return x,y

def get_eval(data):
    x=[]
    y=[]
    max_len=max([len(i["triggers"]) for i in data])
    # left pad triggers

    for row in data:
        speakers=row['speakers']
        emotions=row['emotions']
        utterances=row['utterances']
        triggers=row['triggers']
        # trim in front
        while len(triggers)>max_len:
            triggers.pop(0)
        # left pad triggers
        while len(triggers)<max_len:
            triggers.insert(0,-100)


        # print(speakers,emotions,utterances)
        # trim in front
        while len(utterances)>MAX_UTTERANCES:
            utterances.pop(0)
            speakers.pop(0)
            emotions.pop(0)
        
        # print(speakers,emotions,utterances)
        # pad
        while len(utterances)<MAX_UTTERANCES:
            utterances.insert(0,EOS)
            speakers.insert(0,EOS)
            emotions.insert(0,EOS)
        # print(speakers[1],emotions[1],utterances[1])
        s1=[]
        s2=[]
        t=""
        for i in range(MAX_UTTERANCES):
            if(utterances[i]==EOS):
                s1.append(utterances[i])
                s2.append(utterances[i])
                t=EOS
                continue
            if(t==EOS):
                t=""
            # text.append([t,f"{speakers[-1]}:{utterances[-1]}:{emotions[-1]}"])
            s1.append(t)
            s2.append(f"{speakers[-1]}:{utterances[-1]}:{emotions[-1]}")
            t+=f"{speakers[i]}:{utterances[i]}:{emotions[i]} "
        x.append([s1,s2])
        trigs=[]
        for i in triggers:
            if(i is None):
                trigs.append(0)
            else:
                trigs.append(int(i))
        y.append(trigs)
    
    return x,y


def load_data(PATH):
    with open(PATH) as f:
        data = json.load(f)
        return data
    
label_encoding = {
    "S":0,
    "surprise":1,
    "fear":2,
    "neutral":3,
    "sadness":4,
    "disgust":5,
    "anger":6,
    "joy":7
}


In [6]:
class Dataset(Dataset):
    def __init__(self, data, tokenizer,labels):
        self.data = data
        self.tokenizer = tokenizer
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
#         print(self.data[idx])
#         dat = "".join(self.data[idx])
        s1,s2 = self.data[idx][0],self.data[idx][1]
        s1_tokenized = self.tokenizer(s1,max_length=MAX_LENGTH,truncation=True,return_tensors="pt",padding="max_length")
        s2_tokenized = self.tokenizer(s2,max_length=MAX_LENGTH,truncation=True,return_tensors="pt",padding="max_length")
        input_ids = torch.cat((s1_tokenized["input_ids"],s2_tokenized["input_ids"]),dim=1).squeeze(0)
        # x_tokenized = self.tokenizer(self.data[idx],max_length=MAX_LENGTH,truncation=True,return_tensors="pt",padding="max_length")
        # input_ids = x_tokenized["input_ids"].squeeze(0)
        # attention_mask = x_tokenized["attention_mask"].squeeze(0)
        return input_ids,torch.tensor(self.labels[idx]).squeeze(0)

In [7]:
class MyModel(nn.Module):
    def __init__(self, input_size=MAX_LENGTH*2, hidden_size=MAX_UTTERANCES, num_linear_layers=MAX_UTTERANCES, linear_size=2):
        super(MyModel, self).__init__()
        

        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=False, num_layers=2)
        
        self.linear_layers1 = nn.ModuleList([nn.Linear(hidden_size, 128) for _ in range(num_linear_layers)])
        self.relu = nn.ReLU()
        self.linear_layers2 = nn.ModuleList([nn.Linear(128, linear_size) for _ in range(num_linear_layers)])
        # self.softmax_layers = nn.ModuleList([nn.Softmax(dim=1) for _ in range(num_linear_layers)])
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
 
        for name, param in self.named_parameters():
            if torch.isnan(param).any():
                print(name, "is nan")
                os.exit(0)

        lstm_out, _ = self.lstm(x)

        # print("Output shape of lstm = ",lstm_out.shape)

        if(torch.isnan(lstm_out).any()):
            print("lstm_out is nan"*10)
            print("NAN "*10)
            exit(0)
            
        softmax_outputs = []
        for i in range(len(self.linear_layers1)):
            linear_output1 = self.linear_layers1[i](lstm_out[:, -1, :])

            relu_output = self.relu(linear_output1)

            linear_output2= self.linear_layers2[i](relu_output)


            # softmax_output = self.softmax_layers[i](linear_output2)
            sigmoid_output = self.sigmoid(linear_output2)
            softmax_outputs.append(sigmoid_output.float())
           
        stacked_tensor = torch.stack(softmax_outputs, dim=1)
        return stacked_tensor
#         return stacked_tensor.clone().detach().requires_grad_(True
modnum=3


In [8]:
import torch.nn as nn

criterion = nn.functional.cross_entropy

def metrics(y_true, y_pred):
    acc=accuracy_score(y_true, y_pred)
    f1_macro=f1_score(y_true, y_pred, average='macro')  
    f1_scores=f1_score(y_true, y_pred, average="weighted")
    return acc, f1_macro, f1_scores


In [9]:
def evaluate(model,val_dataloader,name,metric,weights):

    with torch.no_grad():
        model.eval()
        losses = 0

        preds = []
        actuals = []
        for batch in tqdm(val_dataloader, desc=name,total=len(val_dataloader), leave=False):

                input_ids = batch[0].to(device)
                # attention_mask = batch[1].to(device)
                labels = batch[1].to(device)

                batch_size = input_ids.size(0)
                utt_size = labels.size(1)

                outputs  = model(input_ids.float())

                size_diff = labels.size(1) - outputs.size(1)

                # paddings=torch.zeros(batch_size,size_diff,2).to(device)


                # print(outputs.shape,paddings.shape)
                # predicted = torch.cat((outputs,paddings),dim=1)

                predicted=outputs.view(batch_size*utt_size,-1)
                labels=labels.view(batch_size*utt_size)
                
                # print(outputs.shape,labels.shape)
                # loss = criterion(predicted, labels)
                loss = criterion(predicted, labels, weight=weights)


                loss = loss.to(device)
        #         print("Loss = ",loss.item)
                losses+=loss.item()
    
                _, pred = torch.max(predicted, 1)

                # Flatten the predictions and targets
                predicted_flat = pred.view(-1)
                targets_flat = labels.view(-1)

                preds.extend(predicted_flat.cpu().numpy())
                actuals.extend(targets_flat.cpu().numpy())

                del input_ids
                # del attention_mask
                del labels
                del predicted
                # del predicted_labels
                gc.collect()
                torch.cuda.empty_cache()
            # break
        # print(losses,len(val_dataloader))
        x = losses / len(val_dataloader)
        tqdm.write(f"Avg {name} Loss: {x}")
        gc.collect()
        torch.cuda.empty_cache()
        acc,macro,f1 = metric(actuals,preds)
        tqdm.write(f"Avg {name} Accuracy: {acc}, F1 Macro: {macro}, F1 Scores: {f1}")
        return x,acc,macro,f1,preds

# Loading Overall

In [21]:
loaded_model = torch.load(f"{MODEL_PATH}modelM{modnum}.pth")
loaded_tokenizer = torch.load(f"{MODEL_PATH}tokenizerM{modnum}.pth")

# Loading Checkpoint

In [13]:
loaded_model = MyModel()
loaded_tokenizer = torch.load(f"{MODEL_PATH}tokenizerM{modnum}.pth")
loaded_model.load_state_dict(torch.load(f"{MODEL_PATH}model_{modnum}_10.pth"))
loaded_model.to(device)

MyModel(
  (lstm): LSTM(512, 24, num_layers=2, batch_first=True)
  (linear_layers1): ModuleList(
    (0-23): 24 x Linear(in_features=24, out_features=128, bias=True)
  )
  (relu): ReLU()
  (linear_layers2): ModuleList(
    (0-23): 24 x Linear(in_features=128, out_features=2, bias=True)
  )
  (sigmoid): Sigmoid()
)

In [22]:
x_test, y_test = get_train(load_data(PATH+"val_file.json"))

In [23]:
a=0.5
train_loss=[]
eval_loss=[]
weights=torch.tensor([a,1-a]).to(device)
test_dataset = Dataset(x_test,loaded_tokenizer,y_test)
test_dataloader=DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=False)
loss,acc,macro,f1,preds=evaluate(loaded_model,test_dataloader,"Test",metrics,weights)


Test:   0%|          | 0/2 [00:00<?, ?it/s]

Avg Test Loss: 0.6866208016872406
Avg Test Accuracy: 0.276443258204824, F1 Macro: 0.20287395785407622, F1 Scores: 0.17224457255748207


In [27]:
y_test_list = np.array(y_test).flatten()
preds=np.array(preds).flatten()

In [28]:
dic1={}
dic2={}
for i in range(len(y_test_list)):
    if y_test_list[i] in dic1:
        dic1[y_test_list[i]]+=1
    else:
        dic1[y_test_list[i]]=1
    if preds[i] in dic2:
        dic2[preds[i]]+=1
    else:
        dic2[preds[i]]=1

dic1,dic2

({-100: 12939, 0: 6186, 1: 1107}, {0: 13215, 1: 7017})

In [29]:
p=[]
a=[]
for i in range(len(y_test_list)):
    if(y_test_list[i]!=-100):
        # print(y_test_list[i],preds[i])
        p.append(preds[i])
        a.append(y_test_list[i])

print(classification_report(a,p))
f1_s=f1_score(a,p,average=None)

for label, f1 in enumerate(f1_s):
    print(f"F1 score for label {label}: {f1}")

dic1={}
dic2={}
for i in range(len(p)):
    if p[i] in dic1:
        dic1[p[i]]+=1
    else:
        dic1[p[i]]=1
    if a[i] in dic2:
        dic2[a[i]]+=1
    else:
        dic2[a[i]]=1

dic2,dic1


              precision    recall  f1-score   support

           0       0.86      0.87      0.86      6186
           1       0.22      0.20      0.21      1107

    accuracy                           0.77      7293
   macro avg       0.54      0.54      0.54      7293
weighted avg       0.76      0.77      0.76      7293

F1 score for label 0: 0.8633220775044219
F1 score for label 1: 0.2085661080074488


({0: 6186, 1: 1107}, {1: 1041, 0: 6252})

In [30]:
print(classification_report(y_test_list, preds))
f1_scores = f1_score(y_test_list,preds, average=None)

# Print F1 score for each label
for label, f1 in enumerate(f1_scores):
    print(f"F1 score for label {label}: {f1}")

              precision    recall  f1-score   support

        -100       0.00      0.00      0.00     12939
           0       0.41      0.87      0.55      6186
           1       0.03      0.20      0.06      1107

    accuracy                           0.28     20232
   macro avg       0.15      0.36      0.20     20232
weighted avg       0.13      0.28      0.17     20232

F1 score for label 0: 0.0
F1 score for label 1: 0.5534766249162414
F1 score for label 2: 0.055145248645987195


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
